In [1]:
import minsearch

In [2]:
import json 

with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [3]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [4]:
documents[10]

{'text': 'It depends on your background and previous experience with modules. It is expected to require about 5 - 15 hours per week. [source1] [source2]\nYou can also calculate it yourself using this data and then update this answer.',
 'section': 'General course-related questions',
 'question': 'Course - \u200b\u200bHow many hours per week am I expected to spend on this  course?',
 'course': 'data-engineering-zoomcamp'}

In [5]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [6]:
q = 'the course has already started, can I still enroll?'

In [7]:
index.fit(documents)

In [8]:
boost = {'question': 3.0, 'section': 0.5}

results = index.search(
    query = q, 
    filter_dict = {'course': 'data-engineering-zoomcamp'}, 
    boost_dict = boost, 
    num_results = 5
)

In [9]:
from openai import OpenAI

In [10]:
import os
os.environ['OPENAI_API_KEY'] = 'my-api-key'

In [11]:
client = OpenAI()

In [12]:
response = client.chat.completions.create(
    model = 'gpt-4o', 
    messages=[{"role":"user", "content":q}]
)

In [13]:
response.choices[0].message.content

"Typically, it's possible to enroll in a course after it has started, but the specifics depend on the institution or organization offering the course. Here are a few steps you can take to find out if late enrollment is possible:\n\n1. **Check the Course Website or Catalog:** Some institutions provide information about late enrollment or add/drop periods in their course catalogs or on their websites.\n\n2. **Contact the Instructor:** Reach out directly to the course instructor or professor. They may be able to provide guidance, and in some cases, they have the discretion to allow late enrollments.\n\n3. **Speak with Academic Advising:** Academic advisors can provide information on institutional policies and deadlines related to course enrollment.\n\n4. **Contact the Registrar’s Office:** The registrar’s office will have the most accurate information regarding enrollment deadlines and procedures.\n\n5. **Evaluate the Course Material:** If allowed, review the course syllabus and materials

In [14]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when answering the QUESTION. 
If the context doesn't contain the answer, output NONE

QUESTION: {question}

CONTEXT: {context}
""".strip()

In [15]:
context = ""

for doc in results: 
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer:{doc['text']}\n\n"

In [16]:
print(context)

section: General course-related questions
question: Course - Can I still join the course after the start date?
answer:Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer:Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.

section: General course-related questions
question: Course - When will the course start?
answer:The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with t

In [17]:
prompt = prompt_template.format(question = q, context = context).strip()

In [18]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when answering the QUESTION. 
If the context doesn't contain the answer, output NONE

QUESTION: the course has already started, can I still enroll?

CONTEXT: section: General course-related questions
question: Course - Can I still join the course after the start date?
answer:Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer:Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your fin

In [19]:
response = client.chat.completions.create(
    model = 'gpt-4o', 
    messages=[{"role":"user", "content":prompt}]
)

In [20]:
response.choices[0].message.content

'Yes, you can still enroll in the course after it has started. You are eligible to submit the homeworks, but be mindful of the deadlines for the final projects.'

In [21]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [24]:
search('how do I run kafka?')

[{'text': 'In the project directory, run:\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java',
  'section': 'Module 6: streaming with kafka',
  'question': 'Java Kafka: How to run producer/consumer/kstreams/etc in terminal',
  'course': 'data-engineering-zoomcamp'},
 {'text': "Solution from Alexey: create a virtual environment and run requirements.txt and the python files in that environment.\nTo create a virtual env and install packages (run only once)\npython -m venv env\nsource env/bin/activate\npip install -r ../requirements.txt\nTo activate it (you'll need to run it every time you need the virtual env):\nsource env/bin/activate\nTo deactivate it:\ndeactivate\nThis works on MacOS, Linux and Windows - but for Windows the path is slightly different (it's env/Scripts/activate)\nAlso the virtual environment should be created only to run the python file. Docker images should first all be up and running.",
  'section': 'Module 6: streaming wi

In [28]:
query = 'how do I run kafka?'

search_results = search(query)

In [26]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [30]:
build_prompt(query, search_results)

'You\'re a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.\nUse only the facts from the CONTEXT when answering the QUESTION.\n\nQUESTION: how do I run kafka?\n\nCONTEXT: \nsection: Module 6: streaming with kafka\nquestion: Java Kafka: How to run producer/consumer/kstreams/etc in terminal\nanswer: In the project directory, run:\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n\nsection: Module 6: streaming with kafka\nquestion: Module “kafka” not found when trying to run producer.py\nanswer: Solution from Alexey: create a virtual environment and run requirements.txt and the python files in that environment.\nTo create a virtual env and install packages (run only once)\npython -m venv env\nsource env/bin/activate\npip install -r ../requirements.txt\nTo activate it (you\'ll need to run it every time you need the virtual env):\nsource env/bin/activate\nTo deactivate it:\ndeactivate\nThis works on Ma

In [31]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [32]:
query = 'how do I run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [34]:
answer = rag(query)

In [35]:
print(answer)

To run Kafka, you need to follow the specific instructions provided for your setup. Here are two different methods based on your programming language:

1. **Java Kafka**:
   - Navigate to your project directory.
   - Run the following command:
     ```sh
     java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java
     ```

2. **Python Kafka**:
   - First, you need to create a virtual environment and install the necessary packages.
   - To create the virtual environment:
     ```sh
     python -m venv env
     source env/bin/activate
     pip install -r ../requirements.txt
     ```
   - To activate the virtual environment (you'll need to do this every time):
     ```sh
     source env/bin/activate
     ```
   - To deactivate it:
     ```sh
     deactivate
     ```
   - If you are using Windows, the path for activation will be:
     ```sh
     env/Scripts/activate
     ```
   - Make sure all Docker images are up and running before you run the Pyth

In [36]:
rag('the course has already started, can I still enroll?')

"Yes, you can still enroll in the course after it has started. Even if you haven't registered by the start date, you are still eligible to submit the homework assignments. However, please be mindful of deadlines for final projects to avoid last-minute stress."